In [9]:
#!pip install kafka-python

In [1]:
#import bibliotek
from kafka import KafkaConsumer
import ast
import time
from csv import DictWriter
import uuid
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [2]:
# topic = 'quickstart-events'
# field_names = ['id', 'name', 'age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'deposit']
# consumer = KafkaConsumer(topic, bootstrap_servers=['127.0.0.1:9092'])

In [3]:
topic = 'supertest'
field_names = ['id', 'name', 'age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'deposit']
consumer = KafkaConsumer(topic, bootstrap_servers=['ec2-34-237-48-233.compute-1.amazonaws.com:9092'])

In [4]:
#import modelu
loaded_model = pickle.load(open('predict_model.sav', 'rb'))
enc = pickle.load(open('encoder.sav', 'rb'))

In [5]:
col1 = ['job', 'marital', 'education', 'default', 'housing', 'loan']

In [6]:
#funkcja do przetworzania slownika w wiersze 

def append_dict_as_row(file_name, dict_of_elem, field_names):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        # Add dictionary as wor in the csv
        dict_writer.writerow(dict_of_elem)

In [7]:
for message in consumer:
    print(message.value)
    dict_str = message.value.decode("UTF-8")
    temp = ast.literal_eval(dict_str)
    #prediction
    temp['age'] = int(temp['age'])
    temp_row = pd.Series(temp)[col1].to_numpy().reshape(1, -1)
    X_temp = enc.transform(temp_row)
    X_temp = np.append(X_temp, temp['age'])
    temp['deposit'] = int(loaded_model.predict(X_temp.reshape(1, -1))) #przewidywanie etykiety klienta

    temp['id'] = uuid.uuid4() #generowanie unikalnego identyfikatora 
    append_dict_as_row('clients2.csv', temp, field_names)
    time.sleep(4)

b'{"name": "Imie Nazwisko", "age": "79", "job": "retired", "marital": "divorced", "education": "unknown", "default": "no", "housing": "yes", "loan": "yes"}'


KeyboardInterrupt: 